In [1]:
from ROOT import TFile
from EventToGraph import rtfile_to_datalist, MyDataset
from torch_geometric.loader import DataLoader
from sklearn.utils import shuffle

Welcome to JupyROOT 6.22/02


In [2]:
f_sig = TFile.Open(
    "./SelectorOutput/2017/Skim1E2Mu__/Selector_TTToHcToWA_AToMuMu_MHc130_MA90.root"
)
f_bkg = TFile.Open(
    "./SelectorOutput/2017/Skim1E2Mu__/Selector_TTLL_powheg.root"
)


In [3]:
sig_datalist = rtfile_to_datalist(f_sig, is_signal=True)
bkg_datalist = rtfile_to_datalist(f_bkg, is_signal=False)
datalist = shuffle(sig_datalist + bkg_datalist)
train_dataset = MyDataset(datalist[:11000])
test_dataset = MyDataset(datalist[11000:])

In [4]:
train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)

In [5]:
import torch
from torch.nn import Linear, BatchNorm1d
import torch.nn.functional as F
from torch_geometric.nn import GraphConv, GraphNorm
from torch_geometric.nn import global_mean_pool

# TODOL: Make EdgeConv layer!
class GNN(torch.nn.Module):
    def __init__(self):
        super(GNN, self).__init__()
        self.graphnorm0 = GraphNorm(train_dataset[0].num_node_features)
        self.conv1 = GraphConv(train_dataset[0].num_node_features, 256)
        self.graphnorm1 = GraphNorm(256)
        self.conv2 = GraphConv(256, 256)
        self.graphnorm2 = GraphNorm(256)
        self.conv3 = GraphConv(256, 256)
        self.fc1 = Linear(256, 256)
        self.bn1 = BatchNorm1d(256)
        self.fc2 = Linear(256, 2)
        
    def forward(self, x, edge_index, batch):
        # 1. Obtain node embedding
        x = self.graphnorm0(x)
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.graphnorm1(x)
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.graphnorm2(x)
        x = self.conv3(x, edge_index)
        
        # 2. Readout layer
        x = global_mean_pool(x, batch)
        
        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc1(x)
        x = self.bn1(x)
        x = x.relu()
        x = F.dropout(x, p=0.1, training=self.training)
        x = self.fc2(x)
        
        return F.softmax(x, dim=1)

In [6]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu' 
model = GNN().to(DEVICE)
optimizer = torch.optim.Adadelta(model.parameters(), lr=0.02)
criterion = torch.nn.CrossEntropyLoss()


def train():
    model.train()
    
    for data in train_loader:
        out = model(data.x.to(DEVICE), data.edge_index.to(DEVICE), data.batch.to(DEVICE))
        loss = criterion(out, data.y.to(DEVICE))
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
def test(loader):
    model.eval()
    
    correct = 0
    for data in loader:
        out = model(data.x.to(DEVICE), data.edge_index.to(DEVICE), data.batch.to(DEVICE))
        pred = out.argmax(dim=1)
        correct += int((pred == data.y.to(DEVICE)).sum())
    return correct / len(loader.dataset)

In [7]:
for epoch in range(150):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'[EPOCH {epoch}]\tTrain Acc: {train_acc:.4f},\tTest Acc: {test_acc:.4f}')

[EPOCH 0]	Train Acc: 0.6411,	Test Acc: 0.6383
[EPOCH 1]	Train Acc: 0.6865,	Test Acc: 0.6877
[EPOCH 2]	Train Acc: 0.6999,	Test Acc: 0.6988
[EPOCH 3]	Train Acc: 0.7070,	Test Acc: 0.7072
[EPOCH 4]	Train Acc: 0.7134,	Test Acc: 0.7126
[EPOCH 5]	Train Acc: 0.7196,	Test Acc: 0.7176
[EPOCH 6]	Train Acc: 0.7231,	Test Acc: 0.7208
[EPOCH 7]	Train Acc: 0.7271,	Test Acc: 0.7257
[EPOCH 8]	Train Acc: 0.7300,	Test Acc: 0.7308
[EPOCH 9]	Train Acc: 0.7351,	Test Acc: 0.7343
[EPOCH 10]	Train Acc: 0.7380,	Test Acc: 0.7367
[EPOCH 11]	Train Acc: 0.7395,	Test Acc: 0.7398
[EPOCH 12]	Train Acc: 0.7403,	Test Acc: 0.7413
[EPOCH 13]	Train Acc: 0.7428,	Test Acc: 0.7423
[EPOCH 14]	Train Acc: 0.7423,	Test Acc: 0.7432
[EPOCH 15]	Train Acc: 0.7439,	Test Acc: 0.7443
[EPOCH 16]	Train Acc: 0.7443,	Test Acc: 0.7443
[EPOCH 17]	Train Acc: 0.7469,	Test Acc: 0.7440
[EPOCH 18]	Train Acc: 0.7434,	Test Acc: 0.7454
[EPOCH 19]	Train Acc: 0.7466,	Test Acc: 0.7457
[EPOCH 20]	Train Acc: 0.7474,	Test Acc: 0.7456
[EPOCH 21]	Train Acc: 0